In [1]:
# Import libraries and api keys
import hvplot.pandas
import pandas as pd
import requests

from api_keys import geoapify_key

In [2]:
# Load CSV file created in WeatherPy
city_data_df = pd.read_csv("cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,arraial do cabo,-22.9661,-42.0278,26.27,85,97,8.01,BR,1707866198
1,1,guerrero negro,27.9769,-114.0611,17.99,38,0,5.35,MX,1707866198
2,2,sovetsk,55.0839,21.8785,3.10,82,100,2.23,RU,1707866199
3,3,nuku'alofa,-21.1333,-175.2000,30.19,79,75,8.23,TO,1707866199
4,4,margaret river,-33.9500,115.0667,13.28,78,65,4.89,AU,1707866199


In [3]:
# Plot cities on the holoview plot

map_plot = city_data_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo = True, 
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500
)

map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

In [4]:
# Narrow down cities my personal weather preferences
narrowed_city_df = city_data_df.loc[
    (city_data_df["Max Temp"] < 25) & (city_data_df["Max Temp"] > 21) \
    & (city_data_df["Wind Speed"] < 5.5) \
    & (city_data_df["Cloudiness"] < 80),:
]

# Drop nulls
narrowed_city_df = narrowed_city_df.dropna()

narrowed_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
22,22,chonchi,-42.6128,-73.8092,21.56,52,9,0.58,CL,1707866204
61,61,balikpapan,-1.2675,116.8289,24.55,88,20,2.57,ID,1707866215
109,109,hadibu,12.6500,54.0333,23.99,79,28,4.08,YE,1707866228
151,151,kratie,12.4881,106.0188,22.95,73,39,3.25,KH,1707866241
157,157,sarikei,2.1167,111.5167,23.23,99,77,0.87,MY,1707866243
183,183,bardai,11.6500,7.2333,22.14,10,2,4.35,NG,1707866250
246,246,kerikeri,-35.2268,173.9474,22.52,51,12,3.66,NZ,1707866231
249,249,flying fish cove,-10.4217,105.6791,24.82,94,75,1.03,CX,1707866268
259,259,taltal,-25.4000,-70.4833,21.33,76,54,2.39,CL,1707866271
264,264,darwin,-12.4611,130.8418,24.99,93,75,2.57,AU,1707866243


In [5]:
# Figure out hotel arrangements
hotel_df = narrowed_city_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name"
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
22,chonchi,CL,-42.6128,-73.8092,52,
61,balikpapan,ID,-1.2675,116.8289,88,
109,hadibu,YE,12.6500,54.0333,79,
151,kratie,KH,12.4881,106.0188,73,
157,sarikei,MY,2.1167,111.5167,99,
183,bardai,NG,11.6500,7.2333,10,
246,kerikeri,NZ,-35.2268,173.9474,51,
249,flying fish cove,CX,-10.4217,105.6791,94,
259,taltal,CL,-25.4000,-70.4833,76,
264,darwin,AU,-12.4611,130.8418,93,


In [6]:
# Find the first hotel located within 10,000 metres using the Geoapify API

radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":20
}

print("Starting hotel search")

# Iterate through hotel_df
for index, row in hotel_df.iterrows():
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Build parameters
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make API request
    name_address = requests.get(base_url, params=params)
    print(name_address.status_code)
    
    
    # JSONify
    name_address = name_address.json()
    
    # Grab the first hotel from results
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
        print(f"Hotel found: {name_address['features'][0]['properties']['name']}")
    except (KeyError, IndexError):
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
# Display data
hotel_df

Starting hotel search
200
Hotel found: Hotel Huildín
200
Hotel found: Hotel Grand Senyiur
200
200
Hotel found: Luck Life World Hotel
200
200
200
Hotel found: Avalon Resort
200
Hotel found: Christmas Island Lodge
200
Hotel found: Hostería Taltal
200
Hotel found: Mantra Pantanas Darwin
200
Hotel found: Outback Inn
200
Hotel found: Hotel Colón
200
Hotel found: San Martin
200
Hotel found: Sur Hotel
200
Hotel found: Hôtel Zagado
200
200
200
Hotel found: Rainforest Haven Inn
200
Hotel found: Omal Hotel
200
200
Hotel found: Cititel Express
200
Hotel found: Humaita Quality Hotel
200
200
200
Hotel found: Caja Arequipa
200
Hotel found: Whakatane Hotel
200
200
Hotel found: Casa Sodadi
200
Hotel found: EdenWood guesthouse
200
Hotel found: Hôtel Rio Nunez
200
Hotel found: Hotel del Marqués
200
Hotel found: Hostal Dendal
200
200
Hotel found: Hotel Sentosa


,City,Country,Lat,Lng,Humidity,Hotel Name
22,chonchi,CL,-42.6128,-73.8092,52,Hotel Huildín
61,balikpapan,ID,-1.2675,116.8289,88,Hotel Grand Senyiur
109,hadibu,YE,12.6500,54.0333,79,No hotel found
151,kratie,KH,12.4881,106.0188,73,Luck Life World Hotel
157,sarikei,MY,2.1167,111.5167,99,No hotel found
183,bardai,NG,11.6500,7.2333,10,No hotel found
246,kerikeri,NZ,-35.2268,173.9474,51,Avalon Resort
249,flying fish cove,CX,-10.4217,105.6791,94,Christmas Island Lodge
259,taltal,CL,-25.4000,-70.4833,76,Hostería Taltal
264,darwin,AU,-12.4611,130.8418,93,Mantra Pantanas Darwin


In [7]:
# Plot holoview and show hotels on the map

map_plot = hotel_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo = True, 
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    hover_cols = ["Hotel Name", "Country"]
)

map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)